Connect to PostgreSQL

From the Codespaces terminal:

In [ ]:
psql -h db -p 5432 -U dataScience_user -d data_science

When prompted, enter the password.

If successful, you’ll see:

In [ ]:
data_science=#

Explore the Database

In [ ]:
\dt

Create tables (SQL)

We’ll create tables in the public schema.

List tables:

In [ ]:
-- Ensure MDY date parsing for values like 9/1/2024
SET datestyle TO 'ISO, MDY';

-- Drop in dependency order
DROP TABLE IF EXISTS public.bp_readings;
DROP TABLE IF EXISTS public.medications;
DROP TABLE IF EXISTS public.patients;

CREATE TABLE public.patients (
    patient_id     INTEGER PRIMARY KEY,
    first_name     TEXT NOT NULL,
    last_name      TEXT NOT NULL,
    date_of_birth  DATE,
    sex            TEXT,
    race           TEXT,
    ethnicity      TEXT,
    zip_code       TEXT
);

CREATE TABLE public.medications (
    med_id           INTEGER PRIMARY KEY,
    patient_id       INTEGER NOT NULL REFERENCES public.patients(patient_id),
    medication_name  TEXT NOT NULL,
    dose             TEXT,
    frequency        TEXT,
    start_date       DATE,
    indication       TEXT
);

CREATE TABLE public.bp_readings (
    bp_id         INTEGER PRIMARY KEY,
    patient_id    INTEGER NOT NULL REFERENCES public.patients(patient_id),
    reading_date  DATE,
    systolic      INTEGER,
    diastolic     INTEGER,
    heart_rate    INTEGER,
    source        TEXT
);



Load CSVs using \copy (recommended in Codespaces)

Important: use \copy (backslash-copy), not COPY.
\copy reads files from your current terminal filesystem (your repo), which is what you want in Codespaces.

Paste these into psql:

In [ ]:
\copy public.patients FROM 'data/patients.csv' WITH (FORMAT csv, HEADER true);

\copy public.medications FROM 'data/medications.csv' WITH (FORMAT csv, HEADER true);

\copy public.bp_readings FROM 'data/bp_readings.csv' WITH (FORMAT csv, HEADER true);


Verify the load

In [ ]:
SELECT COUNT(*) AS patients_n FROM public.patients;
SELECT COUNT(*) AS medications_n FROM public.medications;
SELECT COUNT(*) AS bp_readings_n FROM public.bp_readings;

SELECT * FROM public.patients LIMIT 5;
SELECT * FROM public.bp_readings LIMIT 5;


Create the cardio10 table

Paste all of this into psql:

In [ ]:
DROP TABLE IF EXISTS public.cardio;

CREATE TABLE public.cardio (
    id                        INTEGER PRIMARY KEY,
    age_days                  INTEGER,
    sex                       INTEGER,
    height                    INTEGER,
    weight                    INTEGER,
    sbp                       INTEGER,
    diastolic                 INTEGER,
    cholesterol               INTEGER,
    gluc                      INTEGER,
    smoking                   INTEGER,
    alco                      INTEGER,
    active                    INTEGER,
    cardio                    INTEGER,

    age                       DOUBLE PRECISION,  -- float (e.g., 50.36)
    bmi                       DOUBLE PRECISION,

    tc                        INTEGER,
    hdl                       INTEGER,
    dm                        INTEGER,
    egfr                      INTEGER,
    bptreat                   INTEGER,
    statin                    INTEGER,

    uacr                      DOUBLE PRECISION,
    sdi                       INTEGER,
    hba1c                     DOUBLE PRECISION,

    prevent_full_10yr_CVD     DOUBLE PRECISION,
    prevent_full_10yr_ASCVD   DOUBLE PRECISION,
    prevent_full_10yr_HF      DOUBLE PRECISION,
    prevent_full_30yr_CVD     DOUBLE PRECISION,
    prevent_full_30yr_ASCVD   DOUBLE PRECISION,
    prevent_full_30yr_HF      DOUBLE PRECISION
);



Load the CSV (treat NA as NULL)

Still inside psql:

In [ ]:
\copy public.cardio10
FROM 'data/cardio10.csv'
WITH (FORMAT csv, HEADER true, NULL 'NA');


Verify

In [ ]:
SELECT COUNT(*) FROM public.cardio10;

SELECT id, age, bmi, prevent_full_10yr_CVD
FROM public.cardio10
LIMIT 5;

SELECT cardio, COUNT(*)
FROM public.cardio10
GROUP BY cardio
ORDER BY cardio;

-- quick missingness check (example)
SELECT
  SUM(CASE WHEN hba1c IS NULL THEN 1 ELSE 0 END) AS hba1c_nulls,
  SUM(CASE WHEN uacr  IS NULL THEN 1 ELSE 0 END) AS uacr_nulls
FROM public.cardio10;


In [ ]:
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';


Exit psql:

In [ ]:
\q

Example Query (Terminal)

In [ ]:
SELECT *
FROM patients
LIMIT 5;


SQL Inside Jupyter Notebooks
2.1 Why Use SQL in Jupyter?

Jupyter is ideal when you want to:

Combine SQL + Python

Store queries alongside explanations

Visualize results

Transition into machine learning or statistics

2.2 Notebook Setup Cell (Run First)

In [9]:
from sqlalchemy import create_engine, text
import pandas as pd

engine = create_engine(
    "postgresql+psycopg2://dataScience_user:data_science@db:5432/data_science",
    pool_pre_ping=True,
)

engine


Engine(postgresql+psycopg2://dataScience_user:***@db:5432/data_science)

Run a SQL Query in Jupyter

In [5]:
query = """
SELECT *
FROM cardio10
LIMIT 5;
"""

df = pd.read_sql(query, engine)
df


,id,age_days,sex,height,weight,sbp,diastolic,cholesterol,gluc,smoking,...,dm,egfr,bptreat,statin,uacr,sdi,hba1c,prevent_full_10yr_cvd,prevent_full_10yr_ascvd,prevent_full_10yr_hf
0,57792,14455,0,164,62,110,70,0,0,0,...,0,88,1,1,0.5,2,6.5,0.199410,0.657339,0.199410
1,17274,15452,0,156,48,110,70,0,0,0,...,0,80,0,1,1.0,1,5.4,0.208530,0.762402,0.208530
2,59931,15235,1,168,72,110,80,1,0,0,...,0,109,0,1,6.7,1,5.6,0.212875,0.828295,0.212875
3,72971,15370,1,178,78,110,70,0,0,0,...,0,100,0,1,4.8,1,5.6,0.215769,0.540801,0.215769
4,33519,15184,0,153,63,110,70,0,0,0,...,0,81,0,0,2.2,4,5.1,0.217907,0.323234,0.217907


SQL runs in the database, but results are returned as a pandas DataFrame.

Example: Filtering Data

In [10]:
query = """
SELECT id, age, sex
FROM cardio10
WHERE age >= 60;
"""

pd.read_sql(query, engine)



,id,age,sex
0,50407,61.60,0
1,2194,64.31,1
2,69113,60.05,1
3,95183,60.37,0
4,48520,62.21,1
...,...,...,...
12241,27150,61.54,1
12242,68549,62.32,1
12243,76392,63.30,0
12244,47101,64.18,0


Part 3: Comparing Terminal vs Jupyter SQL
| Task                     | Terminal (psql) | Jupyter |
| ------------------------ | --------------- | ------- |
| Create tables            | ✅ Best          | ❌       |
| Load CSV data            | ✅ Best          | ❌       |
| Quick inspection         | ✅               | ✅       |
| Reproducible analysis    | ❌               | ✅ Best  |
| Visualization            | ❌               | ✅       |
| Teaching & documentation | ❌               | ✅ Best  |


SQL + Python Together (Power Move)
4.1 Use SQL for Filtering, Python for Analysis

In [11]:
query = """
SELECT sbp, bmi
FROM cardio10
WHERE sbp IS NOT NULL
AND bmi IS NOT NULL;
"""

df = pd.read_sql(query, engine)
df.head()


,sbp,bmi
0,110,23.05
1,110,19.72
2,110,25.51
3,110,24.62
4,110,26.91


Now analyze in Python:

In [ ]:
df.describe()

Part 5: Common Errors & Fixes
❌ psql: command not found

→ PostgreSQL client not installed
→ Use the provided Codespaces environment

❌ password authentication failed

→ Credentials do not match docker-compose.yml

❌ NameError in Jupyter

→ Setup cell not run
→ Restart kernel → Run all

SELECT with a WHERE Clause
To retrieve information about patients who are over 60 years old:


SELECT with a WHERE Clause
To retrieve information about patients who are over 60 years old:


In [14]:
query = """
SELECT *
FROM patients;
"""